In [ ]:
!pip install keras_tuner

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [ ]:
fashion_mnist = keras.datasets.fashion_mnist

In [ ]:
(train_images, train_labels),(test_images,test_labels) = fashion_mnist.load_data()

In [ ]:
train_images = train_images / 255.0
test_images = test_images / 255.0

In [ ]:
train_images[0].shape

(28, 28)

In [ ]:
train_images = train_images.reshape(len(train_images),28,28,1)
test_images = test_images.reshape(len(test_images),28,28,1)

In [ ]:
def build_model(hp):
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters = hp.Int('conv_1_filter', min_value = 32, max_value=128, step = 16),
        kernel_size = hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters = hp.Int('conv_2_filter', min_value = 32, max_value = 64, step =16),
        kernel_size = hp.Choice('conv_2_kernel', values = [3,5]),
        activation = 'relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax') #output layer

  ])

  model.compile(optimizer = keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
  return model

In [ ]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [ ]:
tuner_search = RandomSearch(build_model, 
                            objective = 'val_accuracy', 
                            max_trials = 5, directory = 'output', project_name = "Mnist Fashion")

In [ ]:
tuner_search.search(train_images,train_labels, epochs = 3, validation_split = 0.1)

Trial 6 Complete [00h 16m 22s]
val_accuracy: 0.8533333539962769

Best val_accuracy So Far: 0.9101666808128357
Total elapsed time: 01h 07m 57s
INFO:tensorflow:Oracle triggered exit


In [51]:
model = tuner_search.get_best_models(num_models=1)[0]

In [53]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 64)        640       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 32)        18464     
_________________________________________________________________
flatten (Flatten)            (None, 18432)             0         
_________________________________________________________________
dense (Dense)                (None, 32)                589856    
_________________________________________________________________
dense_1 (Dense)              (None, 10)                330       
Total params: 609,290
Trainable params: 609,290
Non-trainable params: 0
_________________________________________________________________
